In [ ]:
import matplotlib.pyplot
import os
import pathlib
import skimage.io
import skimage.transform
import tensorflow

from metrics import dice_coef, dice_loss, jaccard_distance

In [ ]:
images_predict_folder = os.path.join("dataset", "hcf")
mask_predict_folder = os.path.join("predict", "mask")
pathlib.Path(mask_predict_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
list_images = sorted(list([file for file in pathlib.Path(images_predict_folder).rglob("*")]))

len(list_images)

In [ ]:
model = tensorflow.keras.models.load_model("out/train/unet.h5", custom_objects = {"dice_loss": dice_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })
for i, file in enumerate(list_images):
    print(i, file.resolve())
    img = skimage.io.imread(file.resolve())
    img = skimage.transform.resize(img, (256, 256))
    img = img.reshape((1, 256, 256, 3))
    mask = model.predict(img)
    mask = mask[0, :, :, :]
    filename_mask_pred = os.path.join(mask_predict_folder, str(file.stem) + ".png")
    skimage.io.imsave(filename_mask_pred, skimage.img_as_ubyte(mask))
matplotlib.pyplot.imshow(mask, cmap="gray")